In [1]:
!pip install rdflib

In [2]:
!pip install scikit-learn

In [24]:
#for colab
# !git clone -b OpenKE-PyTorch https://github.com/ozyygen/QuatEOpenKE.git --depth 1

In [5]:
import  pandas as pd
import rdflib
import  numpy as np
import utils

# Storing KG as a DataFrame**

In [6]:
df_cs = utils.load_ttl_files("results-9.ttl")

# Inconsistency checks

Inconsistent connections will be defined and the links will be removed to prepare KG to the next link prediction step. After the link predicition, we expect to see the new links that persist consistency within the hierarchy.

**1. Inconsistency Detection**

1. Clashes between associative links and hierarchical links:

skos:related is a symmetric property, and skos:broaderTransitive and skos:narrowerTransitive are inverses, skos:related is therefore also disjoint with skos:narrowerTransitive:



> **¬∃s, o : skos:related(s, o) ∧ skos:broader(s, o)**



- Example:  
            <ConceptA> skos:broader <ConceptB> ;
            skos:related <ConceptB> .

In the example above, the hierarchy is not consistent with the SKOS data model, because there is a clash between associative links and hierarchical links.

In [7]:
updated_kg, negative_test_df = utils.has_hierarcy_associative_clash(df_cs)
updated_kg.info()
#inconsistent triples -> into the test set
#negative sampling creation -> by considering the inconsistency constraint
#research questions??

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9300 entries, 0 to 9299
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   s       9300 non-null   object
 1   p       9300 non-null   object
 2   o       9300 non-null   object
dtypes: object(3)
memory usage: 218.1+ KB


In [8]:
negative_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   s       56 non-null     object
 1   p       56 non-null     object
 2   o       56 non-null     object
dtypes: object(3)
memory usage: 1.4+ KB


# Dataset Creation
---




1. Train, test and valid sets

Train set is enhanced by True path rule and inverse relation data augmentation. + constraint aware negative sampling

Inverse relations will be added such as < teacher teachTo student > inverse is < student learnBy teacher >

In [10]:
df_train, df_valid, df_test = utils.split_dataset(updated_kg)
df_test. append(negative_test_df, ignore_index=True)

,s,p,o
0,https://semopenalex.org/concept/C46681722,http://www.w3.org/2004/02/skos/core#broader,https://semopenalex.org/concept/C41008148
1,https://semopenalex.org/concept/C35390924,http://www.w3.org/2004/02/skos/core#broader,https://semopenalex.org/concept/C41008148
2,https://semopenalex.org/concept/C2776825360,http://www.w3.org/2004/02/skos/core#broader,https://semopenalex.org/concept/C41008148
3,https://semopenalex.org/concept/C41008148,http://www.w3.org/2004/02/skos/core#related,https://semopenalex.org/concept/C37621935
4,https://semopenalex.org/concept/C106365562,http://www.w3.org/2004/02/skos/core#broader,https://semopenalex.org/concept/C41008148
...,...,...,...
981,https://semopenalex.org/concept/C154945302,http://www.w3.org/2004/02/skos/core#broader,https://semopenalex.org/concept/C41008148
982,https://semopenalex.org/concept/C41008148,http://www.w3.org/2004/02/skos/core#related,https://semopenalex.org/concept/C49774154
983,https://semopenalex.org/concept/C49774154,http://www.w3.org/2004/02/skos/core#broader,https://semopenalex.org/concept/C41008148
984,https://semopenalex.org/concept/C41008148,http://www.w3.org/2004/02/skos/core#related,https://semopenalex.org/concept/C161191863


In [11]:
e_id, r_id = utils.convert_to_id_files(updated_kg)

In [1]:
!pwd

/Users/ozgeerten/Documents/GitHub/QuatEOpenKE/QuatETraining


In [12]:
utils.convert_triples_to_id_files(e_id, r_id,df_train,"train2id.txt")
utils.convert_triples_to_id_files(e_id, r_id,df_test,"test2id.txt")
utils.convert_triples_to_id_files(e_id, r_id,df_valid,"valid2id.txt")

In [14]:
!python n-n.py

In [10]:
#TPR inferred implicit triples
#top_2 = utils.top_2_hierarchy_triples(updated_kg)

# QuatE Implementation
**OpenKE**

In [69]:
!pip3 install torch torchvision

In [70]:
import torch
import os

In [71]:
x = torch.rand(5, 3)
print(x)

tensor([[0.8887, 0.8050, 0.8221],
        [0.5805, 0.5818, 0.1223],
        [0.4709, 0.2965, 0.2407],
        [0.5958, 0.8241, 0.6308],
        [0.5935, 0.3138, 0.3709]])


In [7]:
os.chdir('..')

In [9]:
os.chdir('openke')

In [19]:
#for mac , first time running
# !xcode-select --install

xcode-select: error: command line tools are already installed, use "Software Update" in System Settings to install updates


In [66]:
!bash make.sh

mkdir: release: File exists


In [72]:
#!cp QuatETraining/train_quantE_SemConceptH.py ./
!python train_quantE_SemConceptH.py

Input Files Path : ./benchmarks/FB13/
The toolkit is importing datasets.
The total of relations is 2.
The total of entities is 9301.
The total of train triples is 7440.
Input Files Path : ./benchmarks/FB13/
The total of test triples is 930.
The total of valid triples is 930.
74
Traceback (most recent call last):
  File "/Users/ozgeerten/Documents/GitHub/QuatEOpenKE/openke/train_quantE_SemConceptH.py", line 39, in <module>
    trainer.run()
  File "/Users/ozgeerten/Documents/GitHub/QuatEOpenKE/openke/config/Trainer.py", line 58, in run
    self.model.cuda()
  File "/Users/ozgeerten/miniconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 905, in cuda
    return self._apply(lambda t: t.cuda(device))
  File "/Users/ozgeerten/miniconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 797, in _apply
    module._apply(fn)
  File "/Users/ozgeerten/miniconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 797, in _apply
    module._apply(fn)
  File 